# Roche Capstone - Advanced Model Selection Engine (Systematic Analysis)

## 1. Introduction: Moving Beyond Simple Fitting
This notebook is not just about training a model; it is a **Comparative Analysis** designed to identify the optimal architecture for predicting lab delays in the Roche simulation environment.

We evaluate three classes of models:
1.  **Baseline (Linear Regression)**: To establish a performance floor and check for linear relationships.
2.  **Challenger 1 (Random Forest)**: A robust ensemble method excellent for handling non-linear operational data and outliers.
3.  **Challenger 2 (XGBoost)**: The current 'State of the Art' for tabular data, capable of modeling subtle interactions and handling missing values natively.

Our primary metric is **MAE (Mean Absolute Error)**, as we want to minimize the number of minutes our prediction is off by.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Feature Engineering: The 'Physics' Layer
Raw data is rarely enough. To capture the operational complexities, we engineer interaction terms that reflect reality:
*   **Stress_Index**: A multiplicative interaction of `Scientist_Workload` and `Lab_Occupancy`. High occupancy alone is manageable, but when combined with high workload, it creates a disaster (bottleneck).
*   **Hour_of_Day**: Extracted from timestamps to capture the 'Lunchtime Rush' and daily cycles.

In [ ]:
# Load Data
df_wk = pd.read_csv('workflow_logs.csv')
df_rg = pd.read_csv('reagent_logs.csv')
df_tl = pd.read_csv('telemetry_logs.csv')

# Merge Data Features
df = df_wk.merge(df_rg[['experiment_id', 'reagent_batch_id']], on='experiment_id', how='left')
df = df.merge(df_tl[['experiment_id', 'ambient_temp']], on='experiment_id', how='left')

# --- Feature Engineering ---
# 1. Drift/Aging
df['booking_time'] = pd.to_datetime(df['booking_time'])
start_time = df['booking_time'].min()
df['days_since_start'] = (df['booking_time'] - start_time).dt.days

# 2. Hour of Day (Daily Cycle)
df['hour_of_day'] = df['booking_time'].dt.hour

# 3. Stress Index (Interaction Term)
df['stress_index'] = df['scientist_workload'] * df['lab_occupancy_level']

# 4. Target
df = df.dropna(subset=['delay'])

print("Feature Engineering Complete.")
print(f"New Shape: {df.shape}")
display(df[['stress_index', 'hour_of_day', 'days_since_start']].head())

## 3. The Model Zoo (Benchmarks)
We construct our training pipeline. We will use a standard `ColumnTransformer` to handle preprocessing:
-   **Numerical**: Impute Missing + Scale.
-   **Categorical**: One-Hot Encode (handling unknown categories). 

All models will use the exact same preprocessing to ensure a fair fight.

In [ ]:
# Features Definition
NUM_FEATURES = ['scientist_workload', 'lab_occupancy_level', 'expected_duration', 'ambient_temp', 
                'days_since_start', 'hour_of_day', 'stress_index']
CAT_FEATURES = ['experiment_type', 'instrument_type', 'scientist_experience_level', 'reagent_batch_id']
TARGET = 'delay'

X = df[NUM_FEATURES + CAT_FEATURES]
y = df[TARGET]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())]), NUM_FEATURES),
        ('cat', OneHotEncoder(handle_unknown='ignore'), CAT_FEATURES)
    ])

# Initialize Models
models = {
    "Baseline (Linear Regression)": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

print("Models Initialized.")

## 4. Evaluation & Selection
We evaluate each model using **MAE (Mean Absolute Error)**. 
-   **Why MAE?** It is interpretable. "On average, our prediction is off by X minutes". RMSE is too sensitive to the massive outliers (Gamma tail) in our data.
-   **R2 Score**: Shows how much variance is explained.

In [ ]:
results = []
trained_pipelines = {}

print("Training & Evaluating Models...\n")

for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({"Model": name, "MAE (Min)": mae, "R2 Score": r2})
    trained_pipelines[name] = pipeline
    print(f"{name}: MAE = {mae:.2f} min, R2 = {r2:.4f}")

# Comparison Table
results_df = pd.DataFrame(results).sort_values("MAE (Min)")
display(results_df)

best_model_name = results_df.iloc[0]['Model']
best_pipeline = trained_pipelines[best_model_name]
print(f"\nChampion Model Selected: {best_model_name}")

## 5. Interpretability (The 'Why')
In Pharma/Lab Ops, a black-box model is dangerous. We must understand **what drives the delay**.
We analyze the Feature Importance of the Champion Model to confirm physics-based expectations:
1.  **Is Stress_Index a driver?**
2.  **Are the Bad Batches (e.g., BATCH_392) detected?**

In [ ]:
# Extract Feature Importances
if best_model_name in ['Random Forest', 'XGBoost']:
    model_step = best_pipeline.named_steps['model']
    preproc_step = best_pipeline.named_steps['preprocessor']
    
    # Get feature names from transformers
    cat_names = preproc_step.named_transformers_['cat'].get_feature_names_out(CAT_FEATURES)
    feature_names = NUM_FEATURES + list(cat_names)
    
    importances = model_step.feature_importances_
    
    # Create DF
    imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    imp_df = imp_df.sort_values(by='Importance', ascending=False).head(15)
    
    # Cleanup BATCH names for readability
    imp_df['Feature'] = imp_df['Feature'].apply(lambda x: x.replace('reagent_batch_id_', 'Batch: '))
    
    # Plot
    plt.figure(figsize=(10, 8))
    sns.barplot(x='Importance', y='Feature', data=imp_df, palette='viridis')
    plt.title(f'Feature Importance: Top Drivers of Delay ({best_model_name})')
    plt.xlabel('Importance Score')
    plt.show()
else:
    print("Linear models require coefficients analysis, skipping tree-based importance plot.")

## 6. Productionizing
We save the Champion Model to a pickle file (`lab_delay_model_v2.pkl`) so it can be deployed in the Streamlit App.

In [ ]:
import joblib

# Save the best pipeline
filename = 'lab_delay_model_v2.pkl'
joblib.dump(best_pipeline, filename)
print(f"Champion Model ({best_model_name}) saved to {filename}")